In [10]:
import pandas as pd
import csv
import os

prefix = input("What is the name of the galaxy?")

#real photometry first
real_file = "%s.phot" % prefix

real_df = pd.read_csv(real_file, delim_whitespace=True, header=None) #create data frame 
real_df = real_df.iloc[:,:37] #select till relevant end of data frame (quality flag of I)

#name columns
clmns=['extension','chip','x','y','chi','snr','sharp','round','maj_ax','crowd','type',
           'cts_V','sky_v','nrm_ct_rt_V','nrm_ct_rt_err_V','inst_vega_v','mag_V','Verr','chi_V','snr_v',
           'sharp_V','round_V','crowd_V','flag_V','cts_I','sky_I','nrm_ct_rt_I','nrm_ct_rt_err_I', 
           'inst_vega_I','mag_I','Ierr','chi_I','snr_I','sharp_I','round_I','crowd_I','flag_I']

#to distinguish from fake ones in case we wish to combine these scripts later
string = 'rl_' #for "real"
clmns = [string + x for x in clmns]

#assign columns and preview
real_df.columns=clmns
#real_df



#fake photometry next
fake_file='%s.phot.fake' % prefix #select file 

with open(fake_file) as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    first_row = next(reader)
    num_cols = len(first_row) #get number of columns in file

i = (num_cols - 123) #number of extra columns 
num_EF = 1 + (i/4) #number of images in each filter

fake_df = pd.read_csv(fake_file, delim_whitespace=True, header=None) #create data frame 
fake_df = fake_df.iloc[:,:45+i] #select till relevant end of data frame (quality flag of I)

base_clmns=['extension','chip','x','y','counts_V1','mag_V1','counts_I1', 'mag_I1','extension','chip','x','y',
           'chi','snr','sharp','round','maj_ax','crowd','type','cts_V','sky_v','nrm_ct_rt_V','nrm_ct_rt_err_V',
           'inst_vega_v','mag_V','Verr','chi_V','snr_v','sharp_V','round_V','crowd_V','flag_V','cts_I','sky_I',
           'nrm_ct_rt_I','nrm_ct_rt_err_I', 'inst_vega_I','mag_I','Ierr','chi_I','snr_I','sharp_I','round_I',
           'crowd_I','flag_I'] #base columns

j = 1
k = 1
l = 1

while (j < num_EF):
    m = str(j+1)
    base_clmns.insert(k+5,'counts_V%s' % m) #note- should check this on with both more and less images...
    base_clmns.insert(k+6,'mag_V%s' % m)    
    base_clmns.insert(l+9,'counts_I%s' % m) 
    base_clmns.insert(l+10,'mag_I%s' % m)    
    j = j+1
    k = k+2
    l = l+4
    
if j == num_EF:
    fake_df.columns=base_clmns

#fake_df.iloc[:,4:15] #print relevant columns to see if correct






What is the name of the galaxy?NGC5128-S1
